In [35]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join
from unidecode import unidecode

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime

In [2]:
today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(14)

In [3]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [4]:
mb = Metabase()
diario = mb.download_query('old_diario')
# diario = mb.get_downloaded_query('old_diario')
diario

Cookie:'3508d********-****-****-****-************2602c'

saved_queries: ['all_notifica', 'diario', 'old_diario'] 

tmp_copy:->	 C:\sesa\covid19datascience\input\queries\old_diario_len.csv C:\sesa\covid19datascience\input\queries\tmp\old_diario_len_d060620211125.csv

query_size: 64723

select ... limit 262144 offset 0
Success code 200
tmp_copy:->	 C:\sesa\covid19datascience\input\queries\old_diario_o0.csv C:\sesa\covid19datascience\input\queries\tmp\old_diario_o0_d060620211125.csv
Download finish, time elapsed: 0:00:04.024980
downloaded shape (64723, 22)

Download query time elepsed:00:00:29


'C:\\sesa\\covid19datascience\\input\\queries\\old_diario.csv'

In [5]:
date_parser = lambda x: pd.to_datetime(x, errors='coerce', format='%d/%m/%Y')

notifica = pd.read_csv(diario,
                       sep=',',
                       converters = {
                           'paciente': normalize_text,
                           'origem': lambda x: normalize_number(x,fill=0),
                           'nome_mae': normalize_text,
                           'idade': lambda x: normalize_number(x,fill=0),
                           'pais_residencia': lambda x: normalize_number(x,fill=0),
                           'uf_residencia': lambda x: normalize_number(x,fill=0),
                           'ibge_residencia': lambda x: normalize_number(x,fill=999999),
                           'uf_unidade_notifica': lambda x: normalize_number(x,fill=0),
                           'ibge_unidade_notifica': lambda x: normalize_number(x,fill=999999),
                           'criterio_classificacao': lambda x: normalize_number(x,fill=0),
                           'evolucao': lambda x: normalize_number(x,fill=3) }
                       )


notifica['data_cura_obito'] = notifica['data_cura_obito'].apply(date_parser)
notifica['data_liberacao'] = notifica['data_liberacao'].apply(date_parser)

notifica.loc[notifica['evolucao']==5, 'evolucao'] = 2
notifica.loc[~notifica['evolucao'].isin([1,2,3]),'evolucao'] = 3

notifica['hash'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_residencia'].astype(str) )


notifica['hash_atend'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_unidade_notifica'].astype(str) )

notifica['hash_less'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x - 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_more'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x + 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_diag'] = notifica['paciente'].apply(normalize_hash) + notifica['data_liberacao'].apply(date_hash)

notifica = pd.merge(notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica = pd.merge(notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

notifica = notifica.loc[~(
    (notifica['data_liberacao'].isna()) |
    (notifica['sexo'].isna())
)]

notifica = notifica.loc[(
    (notifica['data_liberacao'] < pd.to_datetime(datetime.today())) &
    (notifica['data_liberacao'] > pd.to_datetime(datetime.strptime('12/03/2020','%d/%m/%Y')))
)]

notifica.groupby('evolucao')[['id']].count()

,id
evolucao,
1,17567
2,821
3,22855


In [6]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")
casosct = cc.df
casosct.tail()

databases: ['cc_04_06_2021', 'cc_05_06_2021', 'cc_06_06_2021']
Load C:\sesa\covid19datascience\bulletin\database\casos_confirmados\cc_05_06_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:59


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_atend,hash_less,hash_more,hash_diag
1115633,1207874,4020936.0,PR,410580,410580,ZILAR MARIA ANTONIOLO DE ALMEIDA,F,54,"COVID-19, Biologia Molecular",2021-06-05,05/06/2021,31/05/2021,3.0,NaT,NaT,ZILARMARIAANTONIOLODEALMEIDA54410580,ZILARMARIAANTONIOLODEALMEIDA54410580,ZILARMARIAANTONIOLODEALMEIDA53410580,ZILARMARIAANTONIOLODEALMEIDA55410580,ZILARMARIAANTONIOLODEALMEIDA05062021
1115634,1207875,3994539.0,PR,412770,412770,ZILDA CUNHA BISPO SANTANA,F,47,"COVID-19, Biologia Molecular",2021-06-04,05/06/2021,29/05/2021,3.0,NaT,NaT,ZILDACUNHABISPOSANTANA47412770,ZILDACUNHABISPOSANTANA47412770,ZILDACUNHABISPOSANTANA46412770,ZILDACUNHABISPOSANTANA48412770,ZILDACUNHABISPOSANTANA04062021
1115635,1207876,4060198.0,PR,412410,412410,ZILDA MOTTA SOBOLEWSKI,F,63,Teste Rápido Antígeno (Imunocromatografia),2021-06-04,05/06/2021,NaN,3.0,NaT,NaT,ZILDAMOTTASOBOLEWSKI63412410,ZILDAMOTTASOBOLEWSKI63412410,ZILDAMOTTASOBOLEWSKI62412410,ZILDAMOTTASOBOLEWSKI64412410,ZILDAMOTTASOBOLEWSKI04062021
1115636,1207877,4059671.0,PR,410480,410480,ZORAIDE ANTUNES RAMOS,F,62,Teste Rápido Antígeno (Imunocromatografia),2021-06-02,05/06/2021,02/06/2021,1.0,2021-06-04 00:00:00,NaT,ZORAIDEANTUNESRAMOS62410480,ZORAIDEANTUNESRAMOS62410480,ZORAIDEANTUNESRAMOS61410480,ZORAIDEANTUNESRAMOS63410480,ZORAIDEANTUNESRAMOS02062021
1115637,1207878,4062081.0,PR,411310,411310,ZULMIRA ROMAO FANTACHOLE,F,78,"COVID-19, Biologia Molecular",2021-06-04,05/06/2021,28/05/2021,3.0,NaT,NaT,ZULMIRAROMAOFANTACHOLE78411310,ZULMIRAROMAOFANTACHOLE78411310,ZULMIRAROMAOFANTACHOLE77411310,ZULMIRAROMAOFANTACHOLE79411310,ZULMIRAROMAOFANTACHOLE04062021


In [7]:
novos_casos = notifica.loc[~(
    (notifica['hash'].isin(casosct['hash'])) |
    (notifica['hash'].isin(casosct['hash_less'])) |
    (notifica['hash'].isin(casosct['hash_more'])) |
    (notifica['hash_diag'].isin(casosct['hash_diag']))
)].copy()

novos_casos['data_com'] = pd.to_datetime(date.today()).strftime('%d/%m/%Y')
novos_casos = novos_casos.sort_values('paciente')
novos_casos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com
44615,3994593,1,01/06/2021,05/06/2021,ABIGLACIL PAULIN DE SOUZA,F,MARIA PAULIN DOS ANJOS,66,29/12/1954,1,41,410940,41,410940,Teste Rápido Antígeno (Imunocromatografia),1,2021-06-05,05/06/2021,05/06/2021,NaN,3,NaT,ABIGLACILPAULINDESOUZA66410940,ABIGLACILPAULINDESOUZA66410940,ABIGLACILPAULINDESOUZA65410940,ABIGLACILPAULINDESOUZA67410940,ABIGLACILPAULINDESOUZA05062021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA,06/06/2021
55376,4037181,1,03/06/2021,06/06/2021,ACIR ARCADIO DIAS,M,NATALIA RODRIGUES,71,12/01/1950,1,41,411070,41,411070,"COVID-19, Biologia Molecular",1,2021-06-06,03/06/2021,04/06/2021,30/05/2021,3,NaT,ACIRARCADIODIAS71411070,ACIRARCADIODIAS71411070,ACIRARCADIODIAS70411070,ACIRARCADIODIAS72411070,ACIRARCADIODIAS06062021,4,IRATI,PR,Irati,IRATI,06/06/2021
44716,3994943,1,01/06/2021,05/06/2021,ACIR LISOSKI,M,MARIA JOANA LISOSKI,55,02/07/1965,1,41,411860,41,411860,"COVID-19, Biologia Molecular",1,2021-06-04,01/06/2021,03/06/2021,30/05/2021,3,NaT,ACIRLISOSKI55411860,ACIRLISOSKI55411860,ACIRLISOSKI54411860,ACIRLISOSKI56411860,ACIRLISOSKI04062021,6,PAULA FREITAS,PR,Paula Freitas,PAULA FREITAS,06/06/2021
57422,4045884,1,04/06/2021,06/06/2021,ACIR SILVEIRA LIMA,M,JUVINA DA SILVEIRA LIMA,55,30/06/1965,1,41,410180,41,410180,"COVID-19, Biologia Molecular",1,2021-06-06,02/06/2021,04/06/2021,31/05/2021,3,NaT,ACIRSILVEIRALIMA55410180,ACIRSILVEIRALIMA55410180,ACIRSILVEIRALIMA54410180,ACIRSILVEIRALIMA56410180,ACIRSILVEIRALIMA06062021,2,ARAUCARIA,PR,Araucária,ARAUCARIA,06/06/2021
36990,3968655,10,31/05/2021,05/06/2021,ADAILTON BECKER,M,YVONE BECKER,50,27/08/1970,1,41,411520,41,411520,"COVID-19, Biologia Molecular",1,2021-05-31,31/05/2021,31/05/2021,28/05/2021,3,NaT,ADAILTONBECKER50411520,ADAILTONBECKER50411520,ADAILTONBECKER49411520,ADAILTONBECKER51411520,ADAILTONBECKER31052021,15,MARINGA,PR,Maringá,MARINGA,06/06/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63194,4067639,1,05/06/2021,05/06/2021,ZENIR DA CRUZ FERREIRA,F,MARIA DA CONCEICAO ANTUNES,65,19/04/1956,1,41,412170,41,412170,Teste Rápido IgM/IgG não discriminatório (Imun...,1,2021-06-05,05/06/2021,NaN,NaN,3,NaT,ZENIRDACRUZFERREIRA65412170,ZENIRDACRUZFERREIRA65412170,ZENIRDACRUZFERREIRA64412170,ZENIRDACRUZFERREIRA66412170,ZENIRDACRUZFERREIRA05062021,21,RESERVA,PR,Reserva,RESERVA,06/06/2021
63158,4067471,1,05/06/2021,05/06/2021,ZENIRA CIVIDINI SILVA,F,IRMA COSSA CIVIDINI,67,08/12/1953,1,41,411550,41,411550,"COVID-19, Biologia Molecular",2,2021-06-05,03/06/2021,05/06/2021,01/06/2021,3,NaT,ZENIRACIVIDINISILVA67411550,ZENIRACIVIDINISILVA67411550,ZENIRACIVIDINISILVA66411550,ZENIRACIVIDINISILVA68411550,ZENIRACIVIDINISILVA05062021,16,MARUMBI,PR,Marumbi,MARUMBI,06/06/2021
26229,3901157,1,27/05/2021,05/06/2021,ZENITA APARECIDA MENDES DA SILVA BEGUER,F,FRANCISCA CAMPOS DA SILVA,54,14/09/1966,1,41,412550,41,412550,"COVID-19, Biologia Molecular",1,2021-05-30,27/05/2021,29/05/2021,24/05/2021,1,2021-06-05,ZENITAAPARECIDAMENDESDASILVABEGUER54412550,ZENITAAPARECIDAMENDESDASILVABEGUER54412550,ZENITAAPARECIDAMENDESDASILVABEGUER53412550,ZENITAAPARECIDAMENDESDASILVABEGUER55412550,ZENITAAPARECIDAMENDESDASILVABEGUER30052021,2,SAO JOSE DOS PINHAIS,PR,São José dos Pinhais,SAO JOSE DOS PINHAIS,06/06/2021
11058,3689718,1,17/05/2021,05/06/2021,ZEQUIAS ANTONIO DE SOUZA,M,LAURA DE CAMARGO SOUZA,49,23/10/1971,1,41,410980,41,410980,"COVID-19, Biologia Molecular",1,2021-05-19,17/05/2021,19/05/2021,14/05/2021,3,NaT,ZEQUIASANTONIODESOUZA49410980,ZEQUIASANTONIODESOUZA49410980,ZEQUIASANTONIODESOUZA48410980,ZEQUIASANTONIODESOUZA50410980,ZEQUIASANTONIODESOUZA1

In [8]:
novos_casos_errados = novos_casos.loc[(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos = novos_casos.loc[~(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame','data_liberacao','data_com','data_1o_sintomas','hash']].to_excel(join(default_output,'novos_casos.xlsx'))

novos_casos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com


In [9]:
obitos_notifica = notifica.loc[notifica['evolucao']==2]
obitos_casos = casosct.loc[casosct['evolucao']==2]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
    (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag'])) 
)]

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casosct['hash'].tolist() + novos_casos['hash'].tolist())]
novos_obitos = novos_obitos.sort_values('paciente')
novos_obitos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
6927,3483599,1,03/05/2021,05/06/2021,ADELINO DE ORNELLAS,M,VILMA DE ORNELLAS JESUS,69,26/02/1952,1,41,410550,41,410550,Vírus Respiratório,1,2021-05-20,17/05/2021,20/05/2021,25/04/2021,2,2021-06-02,ADELINODEORNELLAS69410550,ADELINODEORNELLAS69410550,ADELINODEORNELLAS68410550,ADELINODEORNELLAS70410550,ADELINODEORNELLAS20052021,13,CIANORTE,PR,Cianorte,CIANORTE
22497,3852858,10,25/05/2021,05/06/2021,AILTON SILVEIRA BRUNHOLI,M,BATISTA BRUNHOLI,49,21/06/1971,1,41,411520,41,411520,"COVID-19, Biologia Molecular",1,2021-05-29,29/05/2021,29/05/2021,24/05/2021,2,2021-06-04,AILTONSILVEIRABRUNHOLI49411520,AILTONSILVEIRABRUNHOLI49411520,AILTONSILVEIRABRUNHOLI48411520,AILTONSILVEIRABRUNHOLI50411520,AILTONSILVEIRABRUNHOLI29052021,15,MARINGA,PR,Maringá,MARINGA
64205,4072217,1,05/06/2021,06/06/2021,ANA RODRIGUES DA CRUZ,F,PALMIRA MARIA DE JESUS,83,12/03/1938,1,41,411080,41,411080,"COVID-19, Biologia Molecular",1,2021-05-30,27/05/2021,29/05/2021,01/05/2021,2,2021-06-06,ANARODRIGUESDACRUZ83411080,ANARODRIGUESDACRUZ83411080,ANARODRIGUESDACRUZ82411080,ANARODRIGUESDACRUZ84411080,ANARODRIGUESDACRUZ30052021,11,IRETAMA,PR,Iretama,IRETAMA
26717,3906410,1,27/05/2021,04/06/2021,ANTOINETTE EL ACHKAR,F,LATIFE AL JALKH,82,23/03/1939,1,41,410480,41,410480,"COVID-19, Biologia Molecular",1,2021-05-26,24/05/2021,26/05/2021,20/05/2021,2,2025-06-03,ANTOINETTEELACHKAR82410480,ANTOINETTEELACHKAR82410480,ANTOINETTEELACHKAR81410480,ANTOINETTEELACHKAR83410480,ANTOINETTEELACHKAR26052021,10,CASCAVEL,PR,Cascavel,CASCAVEL
8620,3593798,1,10/05/2021,05/06/2021,ANTONIO FERNANDES MARCHETTI,M,APARECIDA RENOSTO MARCHETTI,58,14/06/1962,1,41,411510,41,411510,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-12,12/05/2021,NaN,06/05/2021,2,2021-05-12,ANTONIOFERNANDESMARCHETTI58411510,ANTONIOFERNANDESMARCHETTI58411510,ANTONIOFERNANDESMARCHETTI57411510,ANTONIOFERNANDESMARCHETTI59411510,ANTONIOFERNANDESMARCHETTI12052021,12,MARILUZ,PR,Mariluz,MARILUZ
16874,3785623,1,20/05/2021,05/06/2021,ANTONIO LUCIANO,M,CATARINA DE MELO LUCIANA,53,10/01/1968,1,41,412630,41,412630,"COVID-19, Biologia Molecular",1,2021-05-23,20/05/2021,22/05/2021,17/05/2021,2,2021-06-05,ANTONIOLUCIANO53412630,ANTONIOLUCIANO53412630,ANTONIOLUCIANO52412630,ANTONIOLUCIANO54412630,ANTONIOLUCIANO23052021,3,SENGES,PR,Sengés,SENGES
23855,3871340,1,26/05/2021,06/06/2021,ARIANE FONTANA DE OLIVEIRA,F,ESTEFANIA GENELI FONTANA,42,06/01/1979,1,41,410480,41,410480,"COVID-19, Biologia Molecular",1,2021-05-26,26/05/2021,26/05/2021,11/05/2021,2,2021-06-06,ARIANEFONTANADEOLIVEIRA42410480,ARIANEFONTANADEOLIVEIRA42410480,ARIANEFONTANADEOLIVEIRA41410480,ARIANEFONTANADEOLIVEIRA43410480,ARIANEFONTANADEOLIVEIRA26052021,10,CASCAVEL,PR,Cascavel,CASCAVEL
10747,3681983,1,15/05/2021,06/06/2021,ARTUR BARRIO,M,CUSTODIA BARRIO CENTRO,50,10/12/1970,1,41,411460,41,999999,Vírus Respiratório,1,2021-05-20,17/05/2021,19/05/2021,12/05/2021,2,2021-06-03,ARTURBARRIO50411460,ARTURBARRIO50999999,ARTURBARRIO49411460,ARTURBARRIO51411460,ARTURBARRIO20052021,20,MARECHAL CANDIDO RONDON,PR,Marechal Cândido Rondon,IGNORADO/EXTERIOR/IG
26194,3900892,1,27/05/2021,05/06/2021,BRUNO BRUCHEZ,M,CERLI TEREZINHA RAMOS,32,23/04/1989,1,41,410500,41,410500,Teste Molecular (RT-PCR),1,2021-05-26,26/05/2021,26/05/2021,25/05/2021,2,2021-06-04,BRUNOBRUCHEZ32410500,BRUNOBRUCHEZ32410500,BRUNOBRUCHEZ31410500,BRUNOBRUCHEZ33410500,BRUNOBRUCHEZ26052021,10,CATANDUVAS,PR,Catanduvas,CATANDUVAS
37845,3971129,1,31/05/2021,06/06/2021,CACILDA DUARTE GUEDES,F,OLIVIA ALVES DUARTE,66,05/08/1954,1,41,410540,41,410540,Vírus Respiratório,1,2021-06-02,31/05/2021,02/06/2021,21/05/2021,2,2021-06-04,CACILDADUARTEGUEDES66410540,CACILDADUARTEGUE

In [10]:
novos_obitos_errados = novos_obitos.loc[(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos = novos_obitos.loc[~(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel(join(default_output,'novos_obitos.xlsx'))

novos_obitos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
26717,3906410,1,27/05/2021,04/06/2021,ANTOINETTE EL ACHKAR,F,LATIFE AL JALKH,82,23/03/1939,1,41,410480,41,410480,"COVID-19, Biologia Molecular",1,2021-05-26,24/05/2021,26/05/2021,20/05/2021,2,2025-06-03,ANTOINETTEELACHKAR82410480,ANTOINETTEELACHKAR82410480,ANTOINETTEELACHKAR81410480,ANTOINETTEELACHKAR83410480,ANTOINETTEELACHKAR26052021,10,CASCAVEL,PR,Cascavel,CASCAVEL
23778,3870060,1,26/05/2021,02/06/2021,JOAO GODOES OLIVEIRA,M,BRANDINA PAULISTA DE GODOES,85,24/06/1935,1,41,411325,41,411325,"COVID-19, Biologia Molecular",1,2021-05-29,25/05/2021,28/05/2021,20/05/2021,2,2021-06-17,JOAOGODOESOLIVEIRA85411325,JOAOGODOESOLIVEIRA85411325,JOAOGODOESOLIVEIRA84411325,JOAOGODOESOLIVEIRA86411325,JOAOGODOESOLIVEIRA29052021,5,LARANJAL,PR,Laranjal,LARANJAL
12023,3710035,1,17/05/2021,02/06/2021,REGINALDO DE LIRA GALINDO,M,ALAIDE PAES LIRA,66,16/12/1954,1,41,411680,41,411680,"COVID-19, Biologia Molecular",1,2021-05-20,17/05/2021,19/05/2021,13/05/2021,2,2021-06-25,REGINALDODELIRAGALINDO66411680,REGINALDODELIRAGALINDO66411680,REGINALDODELIRAGALINDO65411680,REGINALDODELIRAGALINDO67411680,REGINALDODELIRAGALINDO20052021,11,NOVA CANTU,PR,Nova Cantu,NOVA CANTU
11081,3690622,1,17/05/2021,01/06/2021,SEBASTIAO SILVEIRA DE ANDRADE,M,CANTALICIA SILVEIRA DE ANDRADE,68,27/11/1952,1,41,411570,41,411570,"COVID-19, Biologia Molecular",1,2021-05-18,17/05/2021,18/05/2021,14/05/2021,2,2021-06-27,SEBASTIAOSILVEIRADEANDRADE68411570,SEBASTIAOSILVEIRADEANDRADE68411570,SEBASTIAOSILVEIRADEANDRADE67411570,SEBASTIAOSILVEIRADEANDRADE69411570,SEBASTIAOSILVEIRADEANDRADE18052021,1,MATINHOS,PR,Matinhos,MATINHOS
20224,3825589,1,24/05/2021,01/06/2021,VALERIA FRANCISCA DA SILVA,F,AZAIR DA SILVA,42,22/05/1979,1,41,411570,41,411570,"COVID-19, Biologia Molecular",1,2021-05-26,24/05/2021,25/05/2021,NaN,2,2021-06-28,VALERIAFRANCISCADASILVA42411570,VALERIAFRANCISCADASILVA42411570,VALERIAFRANCISCADASILVA41411570,VALERIAFRANCISCADASILVA43411570,VALERIAFRANCISCADASILVA26052021,1,MATINHOS,PR,Matinhos,MATINHOS


In [11]:
casos_confirmados = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios[['ibge','rs']],how='left',on='ibge')

casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

retroativos = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) <= data_retroativos)].sort_values(by='data_liberacao')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_liberacao'].apply(date_parser) <= anteontem)].sort_values(by='data_liberacao')
hoje = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > anteontem)].sort_values(by='data_liberacao')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

relatorio_file = join(default_output,f"relatorio_{(today.strftime('%d/%m/%Y_%Hh_%M').replace('/','_').replace(' ',''))}.txt")
with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{row['idade']}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(row['rs']).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados nas últimas 24 horas.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_liberacao'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_liberacao'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))
        
        try:
            relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        except:
            relatorio.write("Sem novos óbitos retroativos\n")
        try:
            relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        except:
            relatorio.write("Sem novos óbitos ocorridos nas últimas 2 semanas\n")
        try:
            relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos nas últimas 24 horas.\n\n".replace(',','.'))
        except:
            relatorio.write("Sem novos óbitos ocorridos nas últimas 24 horas\n")

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}: {len(value)}\n")

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1115638 + 2685
Total de obitos: 27018 + 50


Total de casos PR: 1109663 + 2678
Total de obitos PR: 26864 + 50
Total de casos Fora: 5975 + 7
Total de obitos Fora: 154 + 0


In [12]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

06/06/2021
2.678 novos casos residentes e 7 não residentes divulgados no PR.
1.112.341 casos confirmados residentes do PR.
1.118.323 total geral.

50 Óbitos residentes do PR:
8 Cascavel
4 Castro
1 Catanduvas
1 Chopinzinho
2 Cianorte
1 Coronel Vivida
1 Céu Azul
4 Foz do Iguaçu
1 Guamiranga
1 Guaratuba
1 Iretama
2 Jacarezinho
1 Jandaia do Sul
1 Lobato
1 Mangueirinha
1 Marechal Cândido Rondon
1 Mariluz
3 Maringá
1 Rio Bonito do Iguaçu
1 Roncador
1 Santa Terezinha de Itaipu
3 Sengés
3 São José dos Pinhais
2 Telêmaco Borba
1 Três Barras do Paraná
2 União da Vitória
1 Vera Cruz do Oeste

26.914 óbitos residentes do PR.
27.068 total geral.

M	69	Cianorte	13	2/jun/2021
M	49	Maringá	15	4/jun/2021
F	83	Iretama	11	6/jun/2021
M	58	Mariluz	12	12/mai/2021
M	53	Sengés	03	5/jun/2021
F	42	Cascavel	10	6/jun/2021
M	50	Marechal Cândido Rondon	20	3/jun/2021
M	32	Catanduvas	10	4/jun/2021
F	66	Chopinzinho	07	4/jun/2021
M	66	Cascavel	10	3/jun/2021
F	70	Castro	03	4/jun/2021
M	64	Cascavel	10	3/jun/2021
F	44	Foz

In [13]:
# while input('bateu?') != 'bateu': pass

In [14]:
# cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")
# casosct = cc.df
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1.0,760586
2.0,27018
3.0,328034


In [15]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

# novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1]),'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['identificacao'] = list(range(casosct.iloc[-1,0] + 1,casosct.iloc[-1,0] + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

1118323


,sexo
evolucao,
1.0,760959
2.0,27018
3.0,330346


In [16]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')

casosct = casosct.set_index('hash')
casosct.update(novos_obitos)
casosct.reset_index(inplace=True)
casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

1118323


,sexo
evolucao,
1.0,760950
2.0,27068
3.0,330305


In [17]:
# exit()
# casosct['is'] = casosct['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
# casosct.loc[casosct['is'].str.contains('/'),'is2'] = casosct.loc[casosct['is'].str.contains('/'),'is'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casosct.loc[casosct['is'].str.contains('-'),'is2'] = casosct.loc[casosct['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casosct['comunicacao'] = casosct['comunicacao'].apply(str)
# casosct.loc[casosct['comunicacao'].str.contains('/'),'com'] = casosct.loc[casosct['comunicacao'].str.contains('/'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casosct.loc[casosct['comunicacao'].str.contains('-'),'com'] = casosct.loc[casosct['comunicacao'].str.contains('-'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casosct['data_evolucao'] = casosct['data_evolucao'].apply(str)
# casosct.loc[casosct['data_evolucao'].str.contains('/'),'evo'] = casosct.loc[casosct['data_evolucao'].str.contains('/'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='coerce'))
# casosct.loc[casosct['data_evolucao'].str.contains('-'),'evo'] = casosct.loc[casosct['data_evolucao'].str.contains('-'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='coerce'))

# casosct['id_notifica'] = casosct['id_notifica'].apply(int)
# casosct['evolucao'] = casosct['evolucao'].apply(int)
# casosct['is'] = casosct['is2']
# casosct['comunicacao'] = casosct['com']
# casosct['data_evolucao'] = casosct['evo'].apply(lambda x: pd.to_datetime(x))

# casosct.dtypes

In [18]:
cc.df = casosct
cc.save(f"cc_{today.strftime('%d_%m_%Y')}",True)
casosct.tail()

Load C:\sesa\covid19datascience\bulletin\database\casos_confirmados\cc_06_06_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:05


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_diag
1118318,1210559,4067639.0,PR,412170,412170,ZENIR DA CRUZ FERREIRA,F,65,Teste Rápido IgM/IgG não discriminatório (Imun...,2021-06-05,06/06/2021,NaN,3.0,NaT,NaT,ZENIRDACRUZFERREIRA65412170,ZENIRDACRUZFERREIRA64412170,ZENIRDACRUZFERREIRA66412170,NaN
1118319,1210560,4067471.0,PR,411550,411550,ZENIRA CIVIDINI SILVA,F,67,"COVID-19, Biologia Molecular",2021-06-05,06/06/2021,01/06/2021,3.0,NaT,NaT,ZENIRACIVIDINISILVA67411550,ZENIRACIVIDINISILVA66411550,ZENIRACIVIDINISILVA68411550,NaN
1118320,1210561,3901157.0,PR,412550,412550,ZENITA APARECIDA MENDES DA SILVA BEGUER,F,54,"COVID-19, Biologia Molecular",2021-05-30,06/06/2021,24/05/2021,1.0,2021-06-05 00:00:00,NaT,ZENITAAPARECIDAMENDESDASILVABEGUER54412550,ZENITAAPARECIDAMENDESDASILVABEGUER53412550,ZENITAAPARECIDAMENDESDASILVABEGUER55412550,NaN
1118321,1210562,3689718.0,PR,410980,410980,ZEQUIAS ANTONIO DE SOUZA,M,49,"COVID-19, Biologia Molecular",2021-05-19,06/06/2021,14/05/2021,3.0,NaT,NaT,ZEQUIASANTONIODESOUZA49410980,ZEQUIASANTONIODESOUZA48410980,ZEQUIASANTONIODESOUZA50410980,NaN
1118322,1210563,1447381.0,PR,411330,411330,ZILEIDE GUERRA,F,20,"COVID-19, Biologia Molecular",2020-11-30,06/06/2021,22/11/2020,1.0,2020-12-20 00:00:00,NaT,ZILEIDEGUERRA20411330,ZILEIDEGUERRA19411330,ZILEIDEGUERRA21411330,NaN


In [19]:
# cc = CasosConfirmados()
# cc.load()
# casosct = cc.df
# casosct.tail()

In [20]:
#recuperados

In [21]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['uf_resid']!='PR','municipio_pr'] = 'Fora'

casosct.loc[casosct['uf_resid']!='PR','ibge7'] = 9999999

casosct['uf_pr'] = 'PR'
casosct.loc[casosct['uf_resid']!='PR','uf_pr'] = 'Fora'

In [22]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

,PR,Fora,GERAL
CASOS,1.112.341,5.982,1.118.323
OBITOS,26.914,154,27.068
RECUPERADOS,758.728,2.222,760.950
ATIVOS,326.699,3.606,330.305


In [23]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
Leste,516.068,13.668,338.048,164.352
Oeste,228.709,4.535,192.404,31.770
Noroeste,180.232,3.832,130.676,45.724
Norte,187.332,4.879,97.600,84.853
Fora,5.982,154,2.222,3.606


In [24]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,regional,,,,
1,Paranaguá,34.878,859,18.171,15.848
2,Metropolitana,329.066,9.508,226.900,92.658
3,Ponta Grossa,65.360,1.561,42.105,21.694
4,Irati,15.850,281,10.976,4.593
5,Guarapuava,35.585,740,20.217,14.628
6,União da Vitória,11.811,226,8.264,3.321
7,Pato Branco,29.898,678,25.359,3.861
8,Francisco Beltrão,40.561,679,33.261,6.621
9,Foz do Iguaçu,55.728,1.192,42.643,11.893


In [43]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

evolucao                                             CASOS  OBITOS  \
macro rs regional      ibge7   municipio_pr                          
Leste 1  01ª Paranaguá 4101200 Antonina              2.338      59   
                       4109500 Guaraqueçaba            636      15   
                       4109609 Guaratuba             4.312     153   
                       4115705 Matinhos              2.805      97   
                       4116208 Morretes              1.987      49   
...                                                    ...     ...   
Norte 22 22ª Ivaiporã  4122172 Rio Branco do Ivaí      261       8   
                       4122651 Rosário do Ivaí         379       2   
                       4123857 Santa Maria do Oeste    662      22   
                       4125001 São João do Ivaí        872      17   
Fora  99 99ª Fora      9999999 Fora                  5.982     154   

evolucao                                             RECUPERADOS  ATIVOS  
macro rs regional      ibge7   municipio_pr                               
Leste 1  01ª Paranaguá 4101200 Antonina                    2.045     234  
                       4109500 Guaraqueçaba                  518     103  
                       4109609 Guaratuba                   3.863     296  
                       4115705 Matinhos                    2.347     361  
                       4116208 Morretes                    1.579     359  
...                                                          ...     ...  
Norte 22 22ª Ivaiporã  4122172 Rio Branco do Ivaí            231      22  
                       4122651 Rosário do Ivaí               359      18  
                       4123857 Santa Maria do Oeste          496     144  
                       4125001 São João do Ivaí              744     111  
Fora  99 99ª Fora      9999999 Fora                        2.222   3.606  

[400 rows x 4 columns]

In [44]:
ministerio_cols = ['rs','municipio_pr','municipio_pr2','macro','CASOS','OBITOS']
total_municipios = total_municipios.reset_index()
total_municipios['municipio_pr2'] = total_municipios['municipio_pr'].apply(unidecode)
ministerio = total_municipios[ministerio_cols].sort_values(['rs','municipio_pr2'])
del ministerio['rs']
del ministerio['municipio_pr2']
ministerio['macro'] = ministerio['macro'].str.upper()

ministerio.loc[399,['municipio_pr']] = 'Fora do Parana'
ministerio.loc[399,['macro']] = ''

ministerio.loc[400,'municipio_pr'] = 'TOTAL GERAL'
ministerio.loc[400,'macro'] = ''
ministerio.loc[400,'CASOS'] = total.loc['CASOS','GERAL']
ministerio.loc[400,'OBITOS'] = total.loc['OBITOS','GERAL']

ministerio.columns = ['Município de residência', 'macro', 'Casos confirmados', 'Óbitos']

ministerio = ministerio.set_index('Município de residência')
ministerio_sort = pd.read_csv(join(default_input,'ministerio.csv'), sep=';').set_index('Município de residência')
ministerio_sort.update(ministerio)
ministerio_sort.reset_index(inplace=True)
ministerio = ministerio_sort

writer = pd.ExcelWriter(join(default_output,"Casos confirmados covid19 no PR por município.xlsx"),
                    engine='xlsxwriter')

ministerio.to_excel(writer,'total_municipios', index=False)
fit_cols(writer,ministerio,'total_municipios')

writer.save()

from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill

wb = load_workbook(filename = join(default_output,"Casos confirmados covid19 no PR por município.xlsx"))
sheet = wb['total_municipios']
sheet.insert_rows(1)
sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=4)
cell = sheet.cell(row=1, column=1)  
cell.value = 'Casos confirmados covid-19 no Paraná'  
cell.alignment = Alignment(horizontal='center', vertical='center')  
wb.save(filename = join(default_output,"Casos confirmados covid19 no PR por município.xlsx"))

ministerio

,Município de residência,macro,Casos confirmados,Óbitos
0,Antonina,LESTE,2.338,59
1,Guaraqueçaba,LESTE,636,15
2,Guaratuba,LESTE,4.312,153
3,Matinhos,LESTE,2.805,97
4,Morretes,LESTE,1.987,49
...,...,...,...,...
396,Rosário do Ivaí,NORTE,379,2
397,Santa Maria do Oeste,NORTE,662,22
398,São João do Ivaí,NORTE,872,17
399,Fora do Parana,,5.982,154


In [27]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [28]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [29]:
writer = pd.ExcelWriter(join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [30]:
writer = pd.ExcelWriter(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

obitos_confirmados.to_excel(writer,'obitos')
fit_cols(writer,obitos_confirmados,'obitos')


writer.save()

In [31]:
writer = pd.ExcelWriter(join(default_output,f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional['macro'] = casos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [32]:
writer = pd.ExcelWriter(join(default_output,f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional['macro'] = obitos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()